##  Pneumonia Pridiction using Convolution Neural Networks

## Author Details:
- Name: Anmol Gupta
- Email: getanmol.gupta@gmail.com
- LinkedIn: www.linkedin.com/in/anmol-gupta-9a905a228

## Prepared For Practice Purpose please hit the upvote if you like the work

## What is Pneumonia?
**Pneumonia is an inflammatory condition of the lung affecting primarily the small air sacs known as alveoli.Symptoms typically include some combination of productive or dry cough, chest pain, fever and difficulty breathing. The severity of the condition is variable. Pneumonia is usually caused by infection with viruses or bacteria and less commonly by other microorganisms, certain medications or conditions such as autoimmune diseases.Risk factors include cystic fibrosis, chronic obstructive pulmonary disease (COPD), asthma, diabetes, heart failure, a history of smoking, a poor ability to cough such as following a stroke and a weak immune system. Diagnosis is often based on symptoms and physical examination. Chest X-ray, blood tests, and culture of the sputum may help confirm the diagnosis.The disease may be classified by where it was acquired, such as community- or hospital-acquired or healthcare-associated pneumonia.**


![Pneumonia Image](https://continentalhospitals.com/uploads/mceu_31923284811699687086478.jpg)


## Description of the Pneumonia Dataset
**The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).
Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients of one to five years old from Guangzhou Women and Children’s Medical Center, Guangzhou. All chest X-ray imaging was performed as part of patients’ routine clinical care.
For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.**

## <div style="text-align: left; background-color:aliceblue ; font-family: Jokerman; color: black; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;">  Importing Required Libraries </div> 

In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image
from skimage import exposure
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras_tuner
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Activation,Dropout,BatchNormalization,InputLayer
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Normal_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/'
Pneumonia_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/PNEUMONIA/'


image_files = [os.path.join(Normal_path, filename) for filename in os.listdir(Normal_path)]
image_files2 = [os.path.join(Pneumonia_path, filename) for filename in os.listdir(Pneumonia_path)]


num_samples = 3

selected_images = random.sample(image_files, num_samples)
selected_images2 = random.sample(image_files2, num_samples)

#create subplots for displaying images
fig, axes = plt.subplots(2, num_samples, figsize=(20,14))

# Display each selected image from the first directory
for i, image_path in enumerate(selected_images):
    ax = axes[0, i]
    img = mpimg.imread(image_path)
    ax.imshow(img)
    ax.set_title('Normal', fontsize=20)  # Increase font size for the title
    ax.axis('on')  # Show axis

# Display each selected image from the second directory
for i, image_path in enumerate(selected_images2):
    ax = axes[1, i]
    img = mpimg.imread(image_path)
    ax.imshow(img)
    ax.set_title('Pneumonia', fontsize=20)  # Increase font size for the title
    ax.axis('on')  # Show axis
    


# Show the images 
plt.tight_layout()  # Adjust layout to prevent overlapping
plt.show()


## <div style="text-align: left; background-color:aliceblue ; font-family: Jokerman; color: black; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;">  EDA  </div> 

In [ ]:
import os

# Path to the training directory
train_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/train/"

# Initialize counters for each class
normal_count = 0
pneumonia_count = 0

# Loop through each class directory and count the number of samples
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        if class_name.lower() == "normal":
            normal_count += len(os.listdir(class_path))
        elif class_name.lower() == "pneumonia":
            pneumonia_count += len(os.listdir(class_path))

# Print the counts for each class
print("Number of samples in Normal class:", normal_count)
print("Number of samples in Pneumonia class:", pneumonia_count)

# Calculate the imbalance ratio
imbalance_ratio = normal_count / pneumonia_count
print("Imbalance Ratio (Normal:Pneumonia):", imbalance_ratio)


In [ ]:
import os
import matplotlib.pyplot as plt

# Path to the training directory
train_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train"

# Initialize counters for each class
normal_count = 0
pneumonia_count = 0

# Loop through each class directory and count the number of samples
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        if class_name.lower() == "normal":
            normal_count += len(os.listdir(class_path))
        elif class_name.lower() == "pneumonia":
            pneumonia_count += len(os.listdir(class_path))

# Create labels and counts for the pie chart
labels = ['Normal', 'Pneumonia']
sizes = [normal_count, pneumonia_count]
colors = ['lightblue', 'lightcoral']
explode = (0, 0.1)  # explode the 2nd slice (i.e., 'Pneumonia')

# Plotting the pie chart
plt.figure(figsize=(8, 6))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Class Distribution in Training Data')
plt.show()


## <div style="text-align: left; background-color:aliceblue ; font-family: Jokerman; color: black; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;">  Preprocessing  </div> 

#### Preprocessing training data

In [ ]:
batch_size = 8

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    directory='/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train',
    target_size=(600, 600),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary'  # or 'categorical' if multiple classes
)

# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val',
        target_size=(600, 600),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'  # or 'categorical' if multiple classes
        )

test_generator = test_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test',
        target_size=(600, 600),
        batch_size=batch_size,
        color_mode='grayscale',
        class_mode='binary'  # or 'categorical' if multiple classes
        )

## <div style="text-align: left; background-color:aliceblue ; font-family: Jokerman; color: black; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;">  Creating CNN model  </div> 

In [ ]:
# create baseline CNN model

model = Sequential()

model.add(Conv2D(256,kernel_size=(3,3),padding='same',activation='relu',input_shape=(600,600,1),kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Conv2D(164,kernel_size=(3,3),padding='same',activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Flatten())

model.add(Dense(164,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())

model.add(Dense(1,activation='sigmoid'))

In [ ]:
from tensorflow.keras.optimizers import Adadelta
optimizer = Adadelta(learning_rate=1.0, rho=0.95)

model.compile(optimizer = optimizer,loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

CALLBACKS = [early_stopping_cb]

In [ ]:
history = model.fit(
    train_generator,
    callbacks=CALLBACKS,
    epochs=100,
    validation_data=test_generator
)

In [ ]:
model.summary()

In [ ]:
Evaluation = model.evaluate(validation_generator)

In [ ]:
Evaluation

In [ ]:
print("Accuracy of this model is :" ,Evaluation[1]*100,"%")

## <div style="text-align: left; background-color:aliceblue ; font-family: Jokerman; color: black; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;">  Accuracy and Loss Graph  </div> 

In [ ]:
plt.plot(history.history['accuracy'], color='red', label='train_accuracy')
plt.plot(history.history['val_accuracy'], color='blue', label='val_accuracy')
plt.legend()
plt.grid()

# Set y-axis limits from 0.1 to 10
plt.ylim(0.1, 1.0)

plt.show()

In [ ]:
plt.plot(history.history['loss'], color = 'red', label = 'train_loss')
plt.plot(history.history['val_loss'], color = 'blue', label = 'val_loss')
plt.legend()
plt.grid()
plt.plot()
plt.show()

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
model.save('Pneumonia_model.h5')

## <div style="text-align: left; background-color:aliceblue ; font-family: Jokerman; color: black; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;">  prediction on random data  </div> 

### Prediction on NormaL Xray

In [ ]:
# Specify the path to the image file
Normal_xray_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg'

# Read the image using imread
Normal_xray_image = mpimg.imread(Normal_xray_path)

# Display the image using imshow
plt.imshow(Normal_xray_image)
plt.show()


In [ ]:
# Specify the path to the image file
Normal_xray_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg'

# Read the image using OpenCV
Normal_xray_image = cv2.imread(Normal_xray_path)

# Convert the image to grayscale
Normal_xray_image_gray = cv2.cvtColor(Normal_xray_image, cv2.COLOR_BGR2GRAY)


# Normalize the resized image
resized_image_normalized = Normal_xray_image_gray / 255.0

# Resize the image to match the desired dimensions
resized_image = cv2.resize(resized_image_normalized, (600, 600))

# Reshape the image to match the expected input shape of the model
resized_image = resized_image.reshape(-1, 600, 600, 1)

# Convert the data type to float32
resized_image = resized_image.astype(np.float32)

# Display the resized grayscale image
plt.imshow(resized_image_normalized, cmap='gray')
plt.axis('on')
plt.show()


In [ ]:
predictions = model.predict(resized_image)

In [ ]:
print((predictions>0.5).astype("int"))

### Prediction on Pneumonia Xray

In [ ]:
# Specify the path to the image file
Pneumonia_xray_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg'

# Read the image using imread
Pneumonia_xray_image = mpimg.imread(Pneumonia_xray_path)

# Display the image using imshow
plt.imshow(Pneumonia_xray_image)
plt.show()


In [ ]:
# Specify the path to the image file
Pneumonia_xray_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/person1004_bacteria_2935.jpeg'

# Read the image using OpenCV
Pneumonia_xray_image = cv2.imread(Pneumonia_xray_path)

# Convert the image to grayscale
Pneumonia_xray_image_gray = cv2.cvtColor(Pneumonia_xray_image, cv2.COLOR_BGR2GRAY)

# Normalize the resized image
Pneumonia_xray_image_normalized = Pneumonia_xray_image_gray / 255.0

# Resize the image to match the desired dimensions
resized_image = cv2.resize(Pneumonia_xray_image_normalized, (600, 600))

# Reshape the image to match the expected input shape of the model
resized_image = resized_image.reshape(-1, 600, 600, 1)

# Convert the data type to float32
resized_image = resized_image.astype(np.float32)

# Display the resized grayscale image
plt.imshow(resized_image_normalized, cmap='gray')
plt.axis('on')
plt.show()


In [ ]:
predictions = model.predict(resized_image)

In [ ]:
print((predictions>0.5).astype("int"))